# Inicio

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("bank_train.csv")
df_clean = df.copy()
df_test = pd.read_csv("bank_test.csv")
df_test_clean = df_test.copy()

## Mapeamento

In [50]:
#Treino
df_clean['y'] = df_clean['y'].replace({"yes": 1, "no": 0})
df_clean['loan'] = df_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
df_clean['default'] = df_clean['default'].replace({'no': 0, 'yes':1})
df_clean['housing'] = df_clean['housing'].replace({'no': 0, 'yes':1})
#Teste
df_test_clean['loan'] = df_test_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
df_test_clean['default'] = df_test_clean['default'].replace({'no': 0, 'yes':1})
df_test_clean['housing'] = df_test_clean['housing'].replace({'no': 0, 'yes':1})

#MAPEAMENTOS
#Treino
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

# Aplicar o mapeamento à coluna 'month'
df_clean['month'] = df_clean['month'].map(month_mapping)

#Teste

month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

# Aplicar o mapeamento à coluna 'month'
df_test_clean['month'] = df_test_clean['month'].map(month_mapping)

education_mapping = {
    'primary': 1, 'secondary': 2, 'tertiary': 3
}

# Aplicar o mapeamento à coluna 'month'
df_clean['education'] = df_clean['education'].map(education_mapping)
#Teste
df_test_clean['education'] = df_test_clean['education'].map(education_mapping)

marital_mapping = {
    'single': 1, 'married': 2, 'divorced': 3
}

df_clean['marital'] = df_clean['marital'].map(marital_mapping)
#Teste
df_test_clean['marital'] = df_test_clean['marital'].map(marital_mapping)

contact_mapping = {
    'cellular': 1, 'unknown': 0, 'telephone': 2
}

df_clean['contact'] = df_clean['contact'].map(contact_mapping)
#Teste
df_test_clean['contact'] = df_test_clean['contact'].map(contact_mapping)

job_mapping = {
    'management':1, 'blue-collar':2, 'student':3, 'self-employed':4, 'technician':5,
 'administrator':6, 'retired':7, 'housemaid':8, 'services':9, 'unemployed':10,
 'admin.':6, 'unknown':0, 'entrepreneur':11 
}

df_clean['job'] = df_clean['job'].map(job_mapping)
#Teste
df_test_clean['job'] = df_test_clean['job'].map(job_mapping)
#aa


C:\Users\jvmil\AppData\Local\Temp\ipykernel_24628\1280514580.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['y'] = df_clean['y'].replace({"yes": 1, "no": 0})
C:\Users\jvmil\AppData\Local\Temp\ipykernel_24628\1280514580.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['loan'] = df_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
C:\Users\jvmil\AppData\Local\Temp\ipykernel_24628\1280514580.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. T

## Limpeza de colunas


In [51]:
#Treino
df_clean = df_clean.drop(columns=['location'])
df_clean = df_clean.drop(columns=['poutcome'])
df_clean = df_clean.drop(columns=['id'])
#Teste
df_test_clean = df_test_clean.drop(columns=['location'])
df_test_clean = df_test_clean.drop(columns=['poutcome'])

In [52]:
df_clean = df_clean[df_clean['job'] != 'unknown']
df_clean = df_clean[df_clean['education'] != 'unknown']


## Remoção de outliers


In [53]:
df_clean= df_clean[(df_clean['balance'] < 15000) & (df_clean['balance'] > -5000) ]
df_clean = df_clean[(df_clean['age'] <= 80) & (df_clean['age'] > 18)]
df_clean = df_clean[(df_clean['previous'] <= 30) ]


In [64]:

df_clean = df_clean.dropna(subset=['marital'])
df_clean = df_clean.dropna(subset=['job'])
df_clean = df_clean.dropna(subset=['contact'])
df_clean = df_clean.dropna(subset=['education'])
df_clean = df_clean.dropna(subset=['previous'])
df_clean = df_clean.dropna(subset=['campaign'])


# Funções auxiliares

## Função de output

In [54]:
def predictions(model):
    X_test_final = df_test_clean.drop(columns=['id'])
    ids = df_test_clean['id']

    # Fazer previsões
    y_pred_final = model.predict(X_test_final)

    # Criar um DataFrame com os resultados
    results = pd.DataFrame({
        'id': ids,
        'y': y_pred_final
    })

    # Salvar os resultados em um arquivo CSV
    results.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')

## Função de threshhold 


In [55]:
def predictions_threshhold(model,valor):
    X_test_final = df_test_clean.drop(columns=['id'])
    ids = df_test_clean['id']

    # Fazer previsões
    y_scores = model.predict_proba(X_test_final)[:, 1]
    y_pred_adjusted = (y_scores >= valor).astype(int)

    # Criar um DataFrame com os resultados
    results = pd.DataFrame({
        'id': ids,
        'y': y_pred_adjusted
    })

    # Salvar os resultados em um arquivo CSV
    results.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')

In [59]:
from sklearn.metrics import precision_score, recall_score, f1_score

def findBestTresh(model, X_test):
    y_scores = model.predict_proba(X_test)[:, 1]

    #valores a serem testados
    
    best_f1 = 0
    best_threshold = 0
    best_precision = 0
    best_recall = 0
    
    for c in range(20,50):
        threshold = c / 100
        y_pred_adjusted = (y_scores >= threshold).astype(int)
        precision = precision_score(y_test, y_pred_adjusted, pos_label=1)
        recall = recall_score(y_test, y_pred_adjusted, pos_label=1)
        f1 = f1_score(y_test, y_pred_adjusted, pos_label=1)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
    
    print(f"Optimal Threshold: {best_threshold}")
    print(f"Best Precision: {best_precision:.2f}")
    print(f"Best Recall: {best_recall:.2f}")
    print(f"Best F1 Score: {best_f1:.2f}")
    
    return best_threshold

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score

def findBestTreshByPrecision(model, X_test):
    y_scores = model.predict_proba(X_test)[:, 1]

    #valores a serem testados
    thresholds = [0.2,0.25,0.27, 0.271, 0.272, 0.273, 0.274, 0.275, 0.276, 0.277, 0.278, 0.279,0.3,0.35,0.4
                  ,0.45,0.5]
    
    best_f1 = 0
    best_threshold = 0
    best_precision = 0
    best_recall = 0
    
    for threshold in thresholds:
        y_pred_adjusted = (y_scores >= threshold).astype(int)
        precision = precision_score(y_test, y_pred_adjusted, pos_label=1)
        recall = recall_score(y_test, y_pred_adjusted, pos_label=1)
        f1 = f1_score(y_test, y_pred_adjusted, pos_label=1)
        
        if precision > best_precision:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
    
    print(f"Optimal Threshold: {best_threshold}")
    print(f"Best Precision: {best_precision:.2f}")
    print(f"Best Recall: {best_recall:.2f}")
    print(f"Best F1 Score: {best_f1:.2f}")
    
    return best_threshold

# Escolha do modelo (lazypredict)

In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)


In [ ]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.90               0.71     0.71      0.89   
DecisionTreeClassifier             0.87               0.71     0.71      0.88   
LGBMClassifier                     0.90               0.69     0.69      0.89   
BaggingClassifier                  0.89               0.67     0.67      0.88   
RandomForestClassifier             0.90               0.66     0.66      0.89   
PassiveAggressiveClassifier        0.78               0.66     0.66      0.81   
AdaBoostClassifier                 0.89               0.65     0.65      0.88   
NearestCentroid                    0.66               0.64     0.64      0.72   
QuadraticDiscriminantAnalysis      0.79               0.64     0.64      0.81   
ExtraTreeClassifier                0.85               0.63     0.63      0.85   
GaussianNB                  

# Testes com o modelo XGBClassifier

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean.drop(columns=['y'])
y = df_clean['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


Normalização dos dados

In [133]:
from sklearn.preprocessing import StandardScaler

cols_to_normalize = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

scaler = StandardScaler()
X_train[cols_to_normalize] = scaler.fit_transform(X_train[cols_to_normalize])
X_test[cols_to_normalize] = scaler.transform(X_test[cols_to_normalize])

KeyError: "['day'] not in index"

Treinando modelo e testando acurácia

In [14]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train, y_train)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)


Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      5660
           1       0.58      0.48      0.52       711

    accuracy                           0.90      6371
   macro avg       0.76      0.72      0.73      6371
weighted avg       0.90      0.90      0.90      6371

Predictions saved to predictions.csv


## Alterando hiperparâmetros

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

param_grid = {
   'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

In [105]:
y_pred_best = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_best)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred_best)
print('Classification Report:\n', report)

predictions(best_model)

Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.95      5635
           1       0.61      0.46      0.53       737

    accuracy                           0.90      6372
   macro avg       0.77      0.71      0.74      6372
weighted avg       0.89      0.90      0.90      6372

Predictions saved to predictions.csv


## Teste com oversampling

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean.drop(columns=['y'])
y_train = df_clean['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

GridSearch para hiperparâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]}

grid_search = GridSearchCV(estimator = xgb, param_grid = grid, cv = 5, scoring = 'f1', verbose = 3, n_jobs = -1)
grid_search.fit(X_train_res, y_train_res)

print("Melhores hiperparâmetros:", grid_search.best_params_)

# Melhor modelo
best_params = grid_search.best_params_

In [ ]:
best_model = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss', **best_params)
best_model.fit(X_train_res, y_train_res)

# Prever no conjunto de teste
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia: %.2f%%" % (accuracy * 100.0))

# Relatório de classificação
print(classification_report(y_test, y_pred))

predictions(best_model)

## Teste deletando a coluna de dias

In [15]:
df_clean_sem_dias = df_clean.copy()
df_clean_sem_dias = df_clean_sem_dias.drop(columns=['day'])

Treino de modelo XGBC

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_sem_dias.drop(columns=['y'])
y = df_clean_sem_dias['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

Teste de previsão

In [19]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train, y_train)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)
print("Coluna sem dias")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Coluna sem dias
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.95      5660
           1       0.59      0.45      0.51       711

    accuracy                           0.90      6371
   macro avg       0.76      0.71      0.73      6371
weighted avg       0.89      0.90      0.90      6371



# Teste separando a idade em faixas

### Alterando as idades em faixas

In [76]:
bins = [0, 24,30, 59, 100]  # Definindo os limites das faixas
labels = [1, 2, 3,4]  # Rótulos para cada faixa
df_clean_faixa_idade = df_clean.copy()
df_clean_faixa_idade['age'] = pd.cut(df_clean_faixa_idade['age'], bins=bins, labels=labels, right=True).astype(int)


Criação do modelo

In [77]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_faixa_idade.drop(columns=['y'])
y = df_clean_faixa_idade['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

### Normalização


In [48]:
df_clean_faixa_idade['balance'] = np.sign(df_clean_faixa_idade['balance']) * np.log1p(np.abs(df_clean_faixa_idade['balance']))

### Oversampling

In [78]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean_faixa_idade.drop(columns=['y'])
y_train = df_clean_faixa_idade['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [68]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train_res, y_train_res)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)

Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      5588
           1       0.62      0.49      0.55       782

    accuracy                           0.90      6370
   macro avg       0.77      0.73      0.75      6370
weighted avg       0.89      0.90      0.90      6370

Predictions saved to predictions.csv


### Teste com probabilidade alterada

In [69]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

#treinar modelo
modelXGBC = XGBClassifier()
modelXGBC.fit(X_train_res, y_train_res)
#threshhold
m = findBestTresh(modelXGBC,X_test)
y_scores = modelXGBC.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = modelXGBC.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)
print("teste")
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(modelXGBC,m)


Optimal Threshold: 0.21
Best Precision: 0.51
Best Recall: 0.77
Best F1 Score: 0.61
Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      5588
           1       0.62      0.49      0.55       782

    accuracy                           0.90      6370
   macro avg       0.77      0.73      0.75      6370
weighted avg       0.89      0.90      0.90      6370

Predictions saved to predictions.csv
teste
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      5588
           1       0.51      0.77      0.61       782

    accuracy                           0.88      6370
   macro avg       0.74      0.83      0.77      6370
weighted avg       0.91      0.88      0.89      6370

Predictions saved to predictions.csv


### Otimização com hiperparâmetros

In [34]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

param_grid = {
   'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [10:58:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Melhores hiperparâmetros: {'learning_rate': 0.02, 'max_depth': 10, 'n_estimators': 100, 'reg_alpha': 0.1}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Hiperparametros rodando 

In [79]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

#treinar modelo
best_model.fit(X_train, y_train)
#threshhold
m = findBestTresh(best_model,X_test)
y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = best_model.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
print("teste")
print(classification_report(y_test, y_pred_adjusted))

predictions_threshhold(best_model,m)


Optimal Threshold: 0.28
Best Precision: 0.56
Best Recall: 0.70
Best F1 Score: 0.62
Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      5588
           1       0.67      0.38      0.48       782

    accuracy                           0.90      6370
   macro avg       0.80      0.68      0.71      6370
weighted avg       0.89      0.90      0.89      6370

teste
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      5588
           1       0.56      0.70      0.62       782

    accuracy                           0.89      6370
   macro avg       0.76      0.81      0.78      6370
weighted avg       0.91      0.89      0.90      6370

Predictions saved to predictions.csv


# Alterando loan e housing

## Clonagem de datapath

In [72]:
df_clean_loan_housing = df_clean.copy()
df_clean_loan_housing.loc[df_clean_loan_housing['housing'] == 1, 'loan'] = 1

In [73]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_loan_housing.drop(columns=['y'])
y = df_clean_loan_housing['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

## Oversampling

In [74]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean_loan_housing.drop(columns=['y'])
y_train = df_clean_loan_housing['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

## Hiperparâmetros

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

param_grid = {
   'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [10:46:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Melhores hiperparâmetros: {'learning_rate': 0.02, 'max_depth': 10, 'n_estimators': 100, 'reg_alpha': 0.0}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [75]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
#threshhold
m = findBestTresh(best_model,X_test)
y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = best_model.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
print("teste")
print(classification_report(y_test, y_pred_adjusted))

predictions_threshhold(best_model,m)

Optimal Threshold: 0.29
Best Precision: 0.50
Best Recall: 0.72
Best F1 Score: 0.59
Coluna faixa idade
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      5588
           1       0.62      0.37      0.46       782

    accuracy                           0.89      6370
   macro avg       0.77      0.67      0.70      6370
weighted avg       0.88      0.89      0.88      6370

teste
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      5588
           1       0.50      0.72      0.59       782

    accuracy                           0.88      6370
   macro avg       0.73      0.81      0.76      6370
weighted avg       0.90      0.88      0.89      6370

Predictions saved to predictions.csv
